In [1]:
from MDXNet import MDXNetDereverb
from infer_uvr5 import _audio_pre_, _audio_pre_new
from config import Config
from tqdm import tqdm
import os
import traceback, pdb
import ffmpeg
import torch
import shutil
import sys

In [2]:
"""代码来源

https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
UVR5模型文件
https://huggingface.co/lj1995/VoiceConversionWebUI/tree/main/uvr5_weights
合格的文件夹路径格式举例： E:\\codes\\py39\\vits_vc_gpu\\白鹭霜华测试样例(去文件管理器地址栏拷就行了)。
模型分为三类：
1、保留人声：不带和声的音频选这个，对主人声保留比HP5更好。内置HP2和HP3两个模型，HP3可能轻微漏伴奏但对主人声保留比HP2稍微好一丁点；
2、仅保留主人声：带和声的音频选这个，对主人声可能有削弱。内置HP5一个模型；
3、去混响、去延迟模型（by FoxJoy）：
  (1)MDX-Net(onnx_dereverb):对于双通道混响是最好的选择，不能去除单通道混响；
  (234)DeEcho:去除延迟效果。Aggressive比Normal去除得更彻底，DeReverb额外去除混响，可去除单声道混响，但是对高频重的板式混响去不干净。
去混响/去延迟，附：
    1、DeEcho-DeReverb模型的耗时是另外2个DeEcho模型的接近2倍；
    2、MDX-Net-Dereverb模型挺慢的；
    3、个人推荐的最干净的配置是先MDX-Net再DeEcho-Aggressive。
"""

'代码来源\n\nhttps://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI\nUVR5模型文件\nhttps://huggingface.co/lj1995/VoiceConversionWebUI/tree/main/uvr5_weights\n合格的文件夹路径格式举例： E:\\codes\\py39\\vits_vc_gpu\\白鹭霜华测试样例(去文件管理器地址栏拷就行了)。\n模型分为三类：\n1、保留人声：不带和声的音频选这个，对主人声保留比HP5更好。内置HP2和HP3两个模型，HP3可能轻微漏伴奏但对主人声保留比HP2稍微好一丁点；\n2、仅保留主人声：带和声的音频选这个，对主人声可能有削弱。内置HP5一个模型；\n3、去混响、去延迟模型（by FoxJoy）：\n\u2003\u2003(1)MDX-Net(onnx_dereverb):对于双通道混响是最好的选择，不能去除单通道混响；\n  (234)DeEcho:去除延迟效果。Aggressive比Normal去除得更彻底，DeReverb额外去除混响，可去除单声道混响，但是对高频重的板式混响去不干净。\n去混响/去延迟，附：\n    1、DeEcho-DeReverb模型的耗时是另外2个DeEcho模型的接近2倍；\n    2、MDX-Net-Dereverb模型挺慢的；\n    3、个人推荐的最干净的配置是先MDX-Net再DeEcho-Aggressive。\n'

In [3]:
from process import uvr_prediction,uvr5_names

In [4]:
# python yuki_builder.py crop的-role_audios输出文件夹 带上voice
dir_wav_input = '/media/checkpoint/speech_data/video/audio/test/voice/'
# 音频说话人的输出文件
opt_vocal_root = '/media/checkpoint/speech_data/video/audio/test/output/'
# 音频伴奏的输出文件
opt_ins_root = '/media/checkpoint/speech_data/video/audio/test/output/'

shutil.rmtree(opt_vocal_root, ignore_errors=True)
os.makedirs(opt_vocal_root, exist_ok=True)

shutil.rmtree(opt_ins_root, ignore_errors=True)
os.makedirs(opt_ins_root, exist_ok=True)
# 人声提取激进程度
agg = 10
# 输出音频格式
format0 = ["wav", "flac", "mp3", "m4a"]

# uvr5_weights文件夹下发的模型，uvr5_names变量中存储
uvr5_names
"""
['onnx_dereverb_By_FoxJoy',
 'HP2_all_vocals',
 'HP2-人声vocals+非人声instrumentals',
 'HP3_all_vocals',
 'HP5_only_main_vocal',
 'HP5-主旋律人声vocals+其他instrumentals',
 'VR-DeEchoAggressive',
 'VR-DeEchoDeReverb',
 'VR-DeEchoNormal']
"""

"\n['onnx_dereverb_By_FoxJoy',\n 'HP2_all_vocals',\n 'HP2-人声vocals+非人声instrumentals',\n 'HP3_all_vocals',\n 'HP5_only_main_vocal',\n 'HP5-主旋律人声vocals+其他instrumentals',\n 'VR-DeEchoAggressive',\n 'VR-DeEchoDeReverb',\n 'VR-DeEchoNormal']\n"

In [5]:
wav_input = '/media/checkpoint/speech_data/video/audio/test/voice/0003_0.00.56.1800_00.00.58.980_为了逃避狼群.wav'
vocal_path, others_path = uvr_prediction(uvr5_names[5], wav_input,
                            opt_vocal_root,
                            opt_ins_root,
                            agg,
                            format0[0]
                            )
print(vocal_path)
print(others_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.10it/s]


0003_0.00.56.1800_00.00.58.980_为了逃避狼群.wav.reformatted.wav instruments done
0003_0.00.56.1800_00.00.58.980_为了逃避狼群.wav.reformatted.wav vocals done
clean_empty_cache
/media/checkpoint/speech_data/video/audio/test/output/vocal_0003_0.00.56.1800_00.00.58.980_为了逃避狼群.wav.reformatted.wav_10.wav
/media/checkpoint/speech_data/video/audio/test/output/instrument_0003_0.00.56.1800_00.00.58.980_为了逃避狼群.wav.reformatted.wav_10.wav
